<h1><span style = "background-color : skyblue"> Generative Adversarial Networks Template </span> </h1>
12 Fabruary, 2021 - <strong>Junyeong Ahn</strong>

In [3]:
import warnings
warnings.filterwarnings(action = "ignore")

import cv2
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

**GPU setting**

In [4]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.5)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))

**Test displaying of mnist data**

In [5]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
x, y = mnist.load_data()

cv2.imshow("MNIST_TEST", x[0][0])
cv2.waitKey(0)

-1

**Define input image dimensions**

In [6]:
img_rows = 28
img_cols = 28
channels = 1 # grayscale
img_shape = (img_rows, img_cols, channels)

<h2>Define generator network</h2>

In [7]:
# Given input of noise (latent) vector, the Generaotr produces an image.
def build_generator():
    
    noise_shape = (100,) # 1D array of size 100 (latent vector / noise)
    
    # Here we are only using Dense layers. But network can be complicated
    # based on the application. For example, you can use VGG for super res. GAN.
    
    model = Sequential()
    
    model.add(Dense(256, input_shape = noise_shape))
    model.add(LeakyReLU(alpha = 0.2)) # 0.2 is from published paper
    model.add(BatchNormalization(momentum = 0.8)) # How fast it trains
    
    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))
    
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha = 0.2)) 
    model.add(BatchNormalization(momentum = 0.8))
    
    model.add(Dense(np.prod(img_shape), activation = "tanh"))
    model.add(Reshape(img_shape))
    
    model.summary()
    
    noise = Input(shape = noise_shape)
    img = model(noise) # Generated image (fake)
    
    return Model(noise, img)

**Given an input image, the Discriminator outputs the likelihood of the image being real.**   
- **Binary classification - true or false (*we're calling it validity)**

<h2>Define discriminator network</h2>

In [8]:
def build_discriminator():
    
    model = Sequential()
    
    model.add(Flatten(input_shape = img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    
    model.add(Dense(256))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dense(1, activation = "sigmoid"))
    
    model.summary()
    
    img = Input(shape = img_shape)
    validity = model(img) # the Discriminator's guess of input being real or not.
    
    return Model(img, validity)

**Now that we have constructed our two models it's time to pit them against each other.**   
   
**We do this by defining a training fuction, loading the data set, re-scaling our training images and setting the ground thrughts.**

<h2>Define train function</h2>

In [9]:
def train(epochs, batch_size = 128, save_interval = 500):
    
    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()
    
    # Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    
    # Add channels dimension. As the input to our gen and discr. has a shape 28x28x1.
    X_train = np.expand_dims(X_train, axis = 3)
    half_batch = int(batch_size / 3)
    

# We then loop through  a number of epochs to train our Discriminator by first selecting
# a random batch of images from our true dataset, generating a set of images from our Generator,
# feeding both set of images into our Discriminator, and finally setting the
# loss parameters for both the real and fake images, as well as the combined loss.

    for epoch in range(epochs):
        
        # ----------------------
        #  Train Discriminator
        # ----------------------
        
        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
        
        noise = np.random.normal(0, 1, (half_batch, 100)) # half_batch x 100 matrix based on Gaussian Distribution
        
        # Generate a half batch of fake images
        gen_imgs = generator.predict(noise)
        
        # Train the discriminator on real and fake images, separately
        # Research showed that separate training is more effective.
        d_loss_real = discriminator.train_on_batch(imgs, np.ones( (half_batch, 1))) # 1 for real images
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros( (half_batch, 1))) # 0 for fake images
        
        # Take average loss from real and fake images.
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        
# And within the same loop we train our Generator, by setting the input noise and
# ultimately training the Generator to have the Discriminator label its samples as valid
# by specifying the gradient loss.
        
        # ----------------------
        #  Train Generator
        # ----------------------
# Create noise vectors as input for generator.
# Create as many noise vectors as defined by the batch size.
# Based on normal distribution. Output will be of size (batch size, 100)
        
        noise = np.random.normal(0, 1, (batch_size, 100))
        
        # The generator wants the discriminator to label the generated samples as valid ones
        # This is where the generator is trying to trick discriminator into believing
        # the generated image is true (hence value of 1 for y)
        valid_y = np.array([1] * batch_size) # Creates an array of all ones of size = batch size
        
        # Generator is part of combined where it got directly linked with the discriminator
        # Train the generator with noise as x and 1 as y.
        # Again, 1 as the output as it is adversarial and if generaotr did a great job
        # of fooling the discriminator then the output would be 1 (true)
        g_loss = combined.train_on_batch(noise, valid_y)
        

# Additionally, in order for us to keep track of our training process, we print the
# progress and save the sameple image output depending on the epoch interval specified.

        # Plot the progress
        #print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], ))
        
        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

**When the specific sample_interval is hit, we call the sample_image function,
which looks as follows.**


<h2>Define image-saver that shows fake image qualities over time</h2>

In [11]:
# This function saves our images for us to view
def save_imgs(epoch):
    
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)
    
    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap = "gray")
            axs[i, j].axis("off")
            cnt += 1
    fig.savefig("images\\mnist_%d.png" % epoch)
    plt.close()

<h2> <span style = "background-color : yellow"> Main Part</span> <h2>

In [12]:
optimizer = Adam(0.0002, 0.5) # Learning rate and momentum

# Build and compile the discriminator first.
# Generator will be trained as part of the compined model, later.
# Pick the loss function and the type of metric to keep track.
# Binary cross entropy as we are doing prediction and it is a better loss function compared to MSE or others.
discriminator = build_discriminator()
discriminator.compile(loss = "binary_crossentropy",
                     optimizer = optimizer,
                     metrics = ["accuracy"])

# Build and compile our Discriminator, pick the loss function
# Since we are only generating (faking) images, let us not track any metrics.
generator = build_generator()
generator.compile(loss = "binary_crossentropy", optimizer = optimizer)

# This builds the Generator and defines the input noise.
# In a GAN the Generator network takes noise z as an input to produce its images.
z = Input(shape = (100,)) # Our random input to the generator
img = generator(z)

# This ensures that when we combine our networks *we only train the Generator*.
# While Generator training we do not want Discriminator weights to be adjusted.
# This dosen't affect the above Discriminator training.
discriminator.trainable = False

# This specifies that our Discriminator will take the images generated by our Generator
# and true dataset and set its output to a parameter called valid, which whill indicate
# whether the input is real or not.
valid = discriminator(img) # Validity check on the generated image

# Here we combined the models and also set our loss function and optimizer.
# Again, we are only training the Generator here.
# The ultimate goal here is for the Generator to fool the Discriminator.
# The combined model (stacked Gen. and Discr.) takes noise as input => generates images => determines validity
combined = Model(z, valid) # Input(z) : image / Output(valid) : validity
combined.compile(loss = "binary_crossentropy", optimizer = optimizer)

train(epochs = 2000, batch_size = 32, save_interval = 20)

# Save model for future use to generate fake images
# Not tested yet..l make sure right model is being saved..
# Compare with GAN4

generator.save("generator_model_test.h5") # Test the model on GAN4_predict...

# Epochs dictate the number of backward and forward propagations, the batch_size
# indicates the number of training smaples per backward/forward propagation, and
# sample_interval specifies after how many epochs we call our sample_image function


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________